# Medir al gestor

In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import date
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios/Gestores'
ruta_asig_baz = '/home/estadistico/Documents/Erick/Banco Azteca/Asignacion csv'
ruta_pagos = '/home/estadistico/Documents/Erick'
mes = 20201201
mes2 = 'Diciembre2020'
mes3 = pd.to_datetime('01/12/2020',dayfirst=True)

In [3]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contrasena = 'estadisticas8474'
base = 'procesos_externos'

In [4]:
#Lectura pagos Bradesco
pagos_brad = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Bradesco/Pagos/ENRIQUEZ Acumulado de pagos Diciembre de 2020.xlsx')
pagos_brad = pagos_brad.iloc[:,0:9]
pagos_brad = pagos_brad.drop_duplicates()
pagos_brad2 = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
pagos_brad2 = pd.DataFrame(pagos_brad2.to_records())
pagos_brad2 = pagos_brad2.loc[pagos_brad2['FECHA_PAGO']>=mes3]

In [5]:
#Lectura pagos Liverpool
sheet_liv = ['SA002REM','SA001REM','DKN04REM','DKN02REM','DKN00REM']
pagos_liv = pd.DataFrame()
for i in sheet_liv:
    pagos_liv_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Liverpool/Pagos/PAGOS DIC LIVERPOOL .xlsx',sheet_name = str(i))
    pagos_liv = pd.concat([pagos_liv,pagos_liv_new],ignore_index=True)
pagos_liv = pagos_liv.drop_duplicates()
pagos_liv = pagos_liv.iloc[:,0:6]
pagos_liv.columns = ['Id','Cuenta','Fecha Prom','Fecha Pago','Monto','Posteo']
try: 
    pagos_liv['Fecha Pago'] = pd.to_datetime(pagos_liv['Fecha Pago'],dayfirst=True)
except: pagos_liv['Fecha Pago']   
pagos_liv['Fecha Pago'] = [datetime.strftime(x,'%d/%m/%Y') for x in pagos_liv['Fecha Pago']]
try: 
    pagos_liv['Fecha Prom'] = pd.to_datetime(pagos_liv['Fecha Prom'],dayfirst=True)
except: pagos_liv['Fecha Prom']    
pagos_liv['Fecha Prom'] = [datetime.strftime(x,'%d/%m/%Y') for x in pagos_liv['Fecha Prom']]
pagos_liv2 = pd.pivot_table(pagos_liv,index=['Id','Cuenta','Fecha Pago','Fecha Prom'],values=['Monto'],aggfunc=np.sum)
pagos_liv2 = pd.DataFrame(pagos_liv2.to_records())
#pagos_liv2 = pagos_liv2.loc[pagos_liv2['Fecha Pago']>=mes3]

In [6]:
sheet_cred = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/')
pagos_cred = pd.DataFrame()
for j in sheet_cred:
    pagos_cred_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/'+j+'')
    pagos_cred = pd.concat([pagos_cred,pagos_cred_new],ignore_index=True)
#pagos_cred.head(3)

In [7]:
sheet_baz = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/')
pagos_baz = pd.DataFrame()
for k in sheet_baz:
    pagos_baz_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/'+k+'')
    pagos_baz = pd.concat([pagos_baz,pagos_baz_new],ignore_index=True)
pagos_baz = pagos_baz.loc[pagos_baz['FNMONTOPAGADO']>0]
pagos_baz = pagos_baz.drop_duplicates(subset=['FNMONTOPAGADO','CAMPANAID','FDFECHAABONO'])
pagos_baz.head(3)

,CALL_CENTER,FDFECHAGENPROMESA,FDFECHAPROMESA,FIIDPERIODO,FNMONTOPROMETIDO,FNMONTOPAGADO,FDFECHAABONO,FNMONTOREQUERIDO,CAMPANAID,CAMPANA,FNSCOMPROMISO,DIASENTREGENYCOMPROMISO,CIERR_SEM,FCEMPNUMCORTE
1,Recuperacion,05/12/2020,08/12/2020,3,176.0,176.0,05/12/2020,206.0,753,Semana 3 Mexico,CUMPLIDO,3,50,467
3,Recuperacion,07/12/2020,09/12/2020,2,963.0,985.0,07/12/2020,910.0,776,Semana 2 Territorio Golfo,CUMPLIDO,2,50,438
5,Recuperacion,05/12/2020,08/12/2020,1,324.0,324.0,06/12/2020,159.0,402,Canales a Terceros de 8 a 14 dias,CUMPLIDO,3,50,520


In [8]:
#Asignamos valores a los parametros \n",
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H-%M')
hoy = np.datetime64('today')

In [9]:
#Hacemos la conexion con el servidor\n",
try: 
    conn = mysql.connector.connect(user=usuario,
                               password=contrasena,
                               host=servidor,
                               port=3306,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

In [10]:
#Funcion para hacer la consulta sql, ya sea de gestion o de asignacion en donde requerimos tres parametros, la tabla de donde vamos a sacar la consulta, en caso de asignacin requerimos los otros dos parametros fecha inicial y fecha final
def consulta_sql(table,from_date=None,to_date=None):
    if from_date is None:
        sql_mes = cursor.callproc(table,)
        for result in cursor.stored_results():
            sql_mes = pd.DataFrame(result.fetchall())
    else:
        sql_mes = cursor.callproc(table, [from_date,to_date])
        for result in cursor.stored_results():
            sql_mes = pd.DataFrame(result.fetchall())
    sql_mes = sql_mes.drop_duplicates()
    return sql_mes

# Reporte diario Liverpool

In [11]:
#Hacemos la consulta referente a las gestiones de Liverpool
gestion_liv = consulta_sql('liverpool_rpt_gestiones_detallado',today,today)
gestion_liv.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv = gestion_liv.loc[gestion_liv['dictamen']=='PROMESA']
gestion_liv['credito'] = pd.to_numeric(gestion_liv['credito'])

In [12]:
#Hacemos la consulta de la asignacion
asignacion_liv = consulta_sql('liverpool_rpt_asignacion_activa')
asignacion_liv = asignacion_liv.iloc[:,[0,1,3,24,6,26]]
asignacion_liv.columns = ['firma_id','unegocio_id','credito','estado','rfc','division']
asignacion_liv['credito'] = pd.to_numeric(asignacion_liv['credito'])

In [13]:
#Hacemos el consolidado 
consolidado_liv = pd.merge(gestion_liv,asignacion_liv,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_liv[cols] = consolidado_liv[cols].apply(pd.to_numeric, errors='coerce')
consolidado_liv = consolidado_liv[(consolidado_liv['monto_promesa']>50)]
consolidado_liv = consolidado_liv[(consolidado_liv['monto_promesa']<500000)]
consolidado_liv.fillna('COBRANZA',inplace=True)

In [14]:
#Obtenemos datos de las promesas de Liverpool
consolidado_liv2 = consolidado_liv.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
prom_liv_gestor = pd.pivot_table(consolidado_liv2,index=['nombre_usuario'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
prom_liv_gestor = pd.DataFrame(prom_liv_gestor.to_records())
prom_liv_gestor = prom_liv_gestor.iloc[:,[0,1,4]]
prom_liv_gestor.columns = ['Gestor','NumPromesas','SumaPromesas']
prom_liv_gestor.fillna(0,inplace=True)
#pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
prom_liv_gestor

,Gestor,NumPromesas,SumaPromesas
0,CRUZ SALAZAR XIMENA,1,1564.0
1,DIAZ DIAZ LEONOR,3,22501.0
2,LOPEZ VASQUEZ YOSELIN,1,770.0
3,LORAYNI GUADALUPE OSNAYA VAZQUEZ,1,2022.0
4,LUCIA ESMERALDA LONGORIA CISNEROS,1,1146.0
5,RODRIGUEZ PAZ DIANA ELIZABETH,1,2600.0
6,TORRES NARVAEZ MAYTE,1,5000.0
7,All,9,35603.0


# Acumulado Mes Liverpool

In [37]:
#Hacemos la consulta referente a las gestiones de Liverpool
gestion_liv_mes = consulta_sql('liverpool_rpt_gestiones_detallado',mes,today)
gestion_liv_mes.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv_mes = gestion_liv_mes.loc[gestion_liv_mes['dictamen']=='PROMESA']
gestion_liv_mes['credito'] = pd.to_numeric(gestion_liv_mes['credito'])
try: gestion_liv_mes['fecha_promesa'] = pd.to_datetime(gestion_liv_mes['fecha_promesa'],dayfirst=False)
except: gestion_liv_mes['fecha_promesa'] = gestion_liv_mes['fecha_gestion']
#gestion_liv_mes.head(3)

In [38]:
#Hacemos el consolidado 
consolidado_liv_mes = pd.merge(gestion_liv_mes,asignacion_liv,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_liv_mes[cols] = consolidado_liv_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']>50)]
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']<500000)]
consolidado_liv_mes.fillna('COBRANZA',inplace=True)
consolidado_liv_mes = consolidado_liv_mes.drop_duplicates(subset = ['usuario','credito','fecha_promesa','monto_promesa'])
#consolidado_liv_mes.head(3)

In [39]:
#Asignamos los aranceles de liverpool
aranceles = [('DKN00REM',0.28),('DKN02REM',0.4),('DKN04REM',0.5),('SA001REM',0.15),('SA002REM',0.16)]
aranceles = pd.DataFrame(aranceles)
aranceles.columns = ['Id','Arancel']
#Agregamos aranceles al consolidado
pagos_liv2 = pd.merge(pagos_liv2,aranceles,how='left',on='Id')
pagos_liv2['PagoArancel'] = [x*y for x,y in zip(pagos_liv2['Monto'],pagos_liv2['Arancel'])]

In [41]:
#Unimos el consolidado con los pagos 
consolidado_pagos_liv = pd.merge(consolidado_liv_mes,pagos_liv2,how='left',left_on=['credito'],right_on=['Cuenta'])
consolidado_pagos_liv = consolidado_pagos_liv.drop_duplicates(subset = ['credito','Monto','usuario','fecha_promesa'])
consolidado_pagos_liv['Fecha Pago'] = pd.to_datetime(consolidado_pagos_liv['Fecha Pago'],dayfirst=True)
consolidado_pagos_liv['dia_prom'] = consolidado_pagos_liv['fecha_promesa'].dt.day
consolidado_pagos_liv['dia_pago'] = consolidado_pagos_liv['Fecha Pago'].dt.day
cols = ['dia_prom','dia_pago','Monto']
consolidado_pagos_liv[cols] = consolidado_pagos_liv[cols].apply(pd.to_numeric, errors='coerce',downcast='integer')
consolidado_pagos_liv['dia_pago'].fillna(0,inplace=True)
try:
    consolidado_pagos_liv['dia_pago'] = [int(x) for x in consolidado_pagos_liv['dia_pago']]
except: consolidado_pagos_liv['dia_pago'] = consolidado_pagos_liv['dia_pago']
#consolidado_pagos_liv.head(3)

In [42]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
consolidado_pagos_liv['PagoReal'] = [np.where((z < x+pd.Timedelta(3, unit='D')) & ((z > x-pd.Timedelta(3, unit='D') )), y, 0) for x,y,z in zip(consolidado_pagos_liv['fecha_promesa'],consolidado_pagos_liv['Monto'],consolidado_pagos_liv['Fecha Pago'])]
consolidado_pagos_liv['PagoReal'] = [x.item() for x in consolidado_pagos_liv['PagoReal']]
#consolidado_pagos_liv.head(3)

In [43]:
consolidado_pagos_liv['MontoArancel'] = [x*y for x,y in zip(consolidado_pagos_liv['PagoReal'],consolidado_pagos_liv['Arancel'])]

In [44]:
#Obtenemos datos de los gestores de Liverpool
prom_liv_gestor_mes = pd.pivot_table(consolidado_pagos_liv,index=['nombre_usuario'],values=['credito','monto_promesa','PagoReal','MontoArancel'],aggfunc=['count',np.sum],margins=True,dropna=False)
prom_liv_gestor_mes = pd.DataFrame(prom_liv_gestor_mes.to_records())
prom_liv_gestor_mes = prom_liv_gestor_mes.iloc[:,[0,3,8,6,5]]
prom_liv_gestor_mes.columns = ['Gestor','Numeropromesas','SumaPromesas','SumaPagos','SumaArancel']
prom_liv_gestor_mes.fillna(0,inplace=True)
prom_liv_gestor_mes = prom_liv_gestor_mes.sort_values(by='SumaPagos',ascending=False)
#prom_liv_gestor_mes[['SumaPromesas']] = prom_liv_gestor_mes[['SumaPromesas']].applymap("{0:.2f}".format)
prom_liv_gestor_mes

,Gestor,Numeropromesas,SumaPromesas,SumaPagos,SumaArancel
26,All,408,1372947.11,818625.41,279132.7974
11,LOPEZ LOPEZ JULIO IVAN,54,237928.55,233364.37,83730.8156
6,GARCIA FLORES OMAR ADRIAN,17,98822.96,98822.96,32551.1998
1,BAUTISTA HILARIO YANELY,16,69908.00,51564.00,14408.1000
7,GARCIA RUIZ ALEXIS,19,42326.00,47862.08,18460.1120
4,DIAZ DIAZ LEONOR,30,131118.00,45640.00,11933.5000
21,RODRIGUEZ MARQUEZ JAVIER,10,59679.00,40812.00,18516.6400
24,TORRES NARVAEZ MAYTE,27,63218.00,38968.00,16124.2000
3,CRUZ SALAZAR XIMENA,29,73243.00,35364.00,12281.1000
13,LORAYNI GUADALUPE OSNAYA VAZQUEZ,28,78987.00,35043.00,12760.8800


In [45]:
pagos_duplicados_liv = consolidado_pagos_liv[consolidado_pagos_liv['Cuenta'].isin(consolidado_pagos_liv['Cuenta'][consolidado_pagos_liv['Cuenta'].duplicated()])]
pagos_duplicados_liv = pagos_duplicados_liv.dropna(subset=['Cuenta']).sort_values('Cuenta')
#pagos_duplicados_liv.head(3)

In [46]:
#Total pagos Liver registrados
pagos_totales_liv = pd.pivot_table(pagos_liv2,index=['Id'],values=['Monto','PagoArancel'],aggfunc=[np.sum,'count'],margins=True)
pagos_totales_liv = pd.DataFrame(pagos_totales_liv.to_records())
pagos_totales_liv = pagos_totales_liv.iloc[:,[0,3,1,2]]
pagos_totales_liv.columns = ['Division','NumeroPagos','SumaPagos','SumaArancel']
pagos_totales_liv.fillna(0,inplace=True)
pagos_totales_liv

,Division,NumeroPagos,SumaPagos,SumaArancel
0,DKN00REM,50,189920.91,53177.8548
1,DKN02REM,76,367387.98,146955.1920
2,DKN04REM,32,178815.95,89407.9750
3,SA001REM,93,154040.80,23106.1200
4,SA002REM,13,24909.26,3985.4816
5,All,264,915074.90,316632.6234


# Medir al gestor Bradesco

# Reporte diario Bradesco

In [25]:
#Hacemos la consulta referente a las gestiones de Bradesco
sql_gest_brad = cursor.callproc('bradescard_rpt_gestiones_detallado', [today,today])
for result in cursor.stored_results():
    gestion_brad = pd.DataFrame(result.fetchall())
gestion_brad.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad = gestion_brad.loc[gestion_brad['dictamen']=='PROMESA']
gestion_brad['credito'] = pd.to_numeric(gestion_brad['credito'])
#gestion_brad.head(3)

In [26]:
#Hacemos la consulta de la asignacion
sql_asig_brad = cursor.callproc('bradescard_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_brad = pd.DataFrame(result.fetchall())
asignacion_brad = asignacion_brad.iloc[:,[0,1,3,9,24,6,26]]
asignacion_brad.columns = ['firma_id','unegocio_id','credito','saldo_total','estado','rfc','division']
asignacion_brad['credito'] = pd.to_numeric(asignacion_brad['credito'])
#asignacion_brad.head(3)

In [27]:
#Hacemos la union de asignacion y promesas
consolidado_brad = pd.merge(gestion_brad,asignacion_brad,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_brad[cols] = consolidado_brad[cols].apply(pd.to_numeric, errors='coerce')
consolidado_brad = consolidado_brad[(consolidado_brad['monto_promesa']>50)]
consolidado_brad = consolidado_brad[(consolidado_brad['monto_promesa']<500000)]
consolidado_brad.fillna('COBRANZA',inplace=True)
#consolidado_brad.head(3)

In [28]:
#Obtenemos datos de las promesas de Bradesco
consolidado_brad2 = consolidado_brad.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
prom_brad_gestor = pd.pivot_table(consolidado_brad2,index=['nombre_usuario'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
prom_brad_gestor = pd.DataFrame(prom_brad_gestor.to_records())
prom_brad_gestor = prom_brad_gestor.iloc[:,[0,1,4]]
prom_brad_gestor.columns = ['Gestor','NumeroPromesas','SumaPromesas']
prom_brad_gestor.fillna(0,inplace=True)
#prom_brad_gestor[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
prom_brad_gestor

,Gestor,NumeroPromesas,SumaPromesas
0,AGUILAR MACEDO VALERIA,2,1432.0
1,BALBUENA MACEDO EDGAR URIEL,1,700.0
2,CASTRO REZA IRVING YAEL,1,3385.0
3,COLULA CALIXTO MIGUEL ANGEL,2,4500.0
4,DANIEL PINEDA ALVARADO,2,3600.0
5,DE LA FUENTE MIJARES BETTINA,2,3530.0
6,DOMINGUEZ HERNANDEZ ANAID ITZEL,4,4350.0
7,GARCIA CHAVEZ BRAYAN,1,500.0
8,MUNOZ BORJA KATYA MUSMED,3,6100.0
9,PEDRAZA COLIN MARIA DEL CONZUELO,1,4385.0


# Acumulado mes Bradesco

In [29]:
#Hacemos la consulta referente a las gestiones de Bradesco
sql_gest_brad_mes = cursor.callproc('bradescard_rpt_gestiones_detallado', [mes,today])
for result in cursor.stored_results():
    gestion_brad_mes = pd.DataFrame(result.fetchall())
gestion_brad_mes.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad_mes = gestion_brad_mes.loc[gestion_brad_mes['dictamen']=='PROMESA']
gestion_brad_mes['credito'] = pd.to_numeric(gestion_brad_mes['credito'])
gestion_brad_mes['monto_promesa'] = pd.to_numeric(gestion_brad_mes['monto_promesa'])
gestion_brad_mes = gestion_brad_mes.drop_duplicates(subset=['credito','usuario','fecha_promesa','monto_promesa'])
#gestion_brad_mes.head(3)

In [30]:
#Hacemos la union de asignacion y promesas
consolidado_brad_mes = pd.merge(gestion_brad_mes,asignacion_brad,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_brad_mes[cols] = consolidado_brad_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']>50)]
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']<500000)]
consolidado_brad_mes.fillna('COBRANZA',inplace=True)
#consolidado_brad_mes.head(3)

In [61]:
#Obtenemos los pagos totales por division
pagos_totales_brad = pd.pivot_table(pagos_brad2,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'],margins=True,dropna=False)
pagos_totales_brad = pd.DataFrame(pagos_totales_brad.to_records())
pagos_totales_brad = pagos_totales_brad.iloc[:,[0,2,1]]
pagos_totales_brad.columns = ['Division','NumeroPagos','SumaPagos']
pagos_totales_brad.fillna(0,inplace=True)
pagos_totales_brad['SumaPagos'] = ['${:,.2f}'.format(x) for x in pagos_totales_brad['SumaPagos']]
pagos_totales_brad

,Division,NumeroPagos,SumaPagos
0,C0 4PV,25,"$56,031.00"
1,C0 5PV,10,"$8,209.00"
2,C0 6PV,8,"$12,240.00"
3,C1,28,"$38,004.24"
4,C2,8,"$8,449.00"
5,C3,21,"$22,699.00"
6,C4,7,"$8,403.00"
7,All,107,"$154,035.24"


In [62]:
def func(row):
    if row['TIPO_ASIGANCION'] == 'C0 4PV':
        return .07
    elif row['TIPO_ASIGANCION'] == 'C0 5PV':
        return .09
    elif row['TIPO_ASIGANCION'] == 'C0 6PV':
        return .1
    elif row['TIPO_ASIGANCION'] == 'A':
        return .27
    elif row['TIPO_ASIGANCION'] == 'B':
        return .25
    elif row['TIPO_ASIGANCION'] == 'C1':
        return .17
    elif row['TIPO_ASIGANCION'] == 'C2':
        return .19
    elif row['TIPO_ASIGANCION'] == 'C3':
        return .21
    elif row['TIPO_ASIGANCION'] == 'C4':
        return .23
    else:
        return 0

pagos_brad2['Arancel'] = pagos_brad2.apply(func, axis=1)

In [63]:
pagos_brad2['Facturacion'] = [x*y for x,y in zip(pagos_brad2['MONTO'],pagos_brad2['Arancel'])]

In [64]:
#Hacemos la union consolidado y pagos
consoliado_pagos_brad = pd.merge(consolidado_brad_mes,pagos_brad2,how='left',left_on=['credito'],right_on=['NUMERO_TARJETA'])
try: consoliado_pagos_brad['fecha_promesa'] = pd.to_datetime(consoliado_pagos_brad['fecha_promesa'],dayfirst=False)
except: consoliado_pagos_brad['fecha_promesa'] = consoliado_pagos_brad['fecha_gestion']

In [65]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
consoliado_pagos_brad['PagoUnico'] = [np.where((z < x+pd.Timedelta(4, unit='D')) & ((z > x-pd.Timedelta(3, unit='D') )), y, 0) for x,y,z in zip(consoliado_pagos_brad['fecha_promesa'],consoliado_pagos_brad['MONTO'],consoliado_pagos_brad['FECHA_PAGO'])]
consoliado_pagos_brad['PagoUnico'] = [x.item() for x in consoliado_pagos_brad['PagoUnico']]

In [66]:
consoliado_pagos_brad['PagoReal'] = [x*y for x,y in zip(consoliado_pagos_brad['PagoUnico'],consoliado_pagos_brad['Arancel'])]
consoliado_pagos_brad2 = consoliado_pagos_brad.drop_duplicates(subset=['credito','MONTO','FECHA_PAGO','usuario'])

In [67]:
#Obtenemos datos de las promesas de Bradesco
prom_brad_gestor_mes = pd.pivot_table(consoliado_pagos_brad2,index='nombre_usuario',values=['credito','monto_promesa','PagoUnico','PagoReal'], aggfunc = ['count',np.sum], margins=True,dropna=False)
prom_brad_gestor_mes = pd.DataFrame(prom_brad_gestor_mes.to_records())
prom_brad_gestor_mes = prom_brad_gestor_mes.iloc[:,[0,4,8,6,5]]
prom_brad_gestor_mes.columns = ['Gestor','NumeroPromesas','SumaPromesas','SumaPago','SumaFacturacion']
prom_brad_gestor_mes = prom_brad_gestor_mes.sort_values(by=['SumaPago'],ascending =False)
prom_brad_gestor_mes = prom_brad_gestor_mes.reset_index(drop=True)
#prom_brad_gestor_mes.fillna(0,inplace=True)
#prom_brad_gestor_mes[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = prom_brad_gestor_mes[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
prom_brad_gestor_mes

,Gestor,NumeroPromesas,SumaPromesas,SumaPago,SumaFacturacion
0,All,448,715729.0,87907.0,13119.79
1,AGUILAR MACEDO VALERIA,27,32771.0,14430.0,2250.74
2,GOMEZ CARRILLO LETICIA,17,38638.0,10333.0,1512.80
3,MIRIAM PEINADO ALVAREZ,19,23820.0,9363.0,1120.45
4,ROSAS TREJO JESUS EDUARDO,9,13662.0,8317.0,1512.05
5,JUAREZ ROBLEDO LORENA,16,22091.0,4735.0,577.71
6,ALEJANDRA YESENIA HERNANDEZ BITE,8,10730.0,4721.0,960.89
7,MANUEL BUSTOS LUCERO,2,5162.0,4588.0,321.16
8,LIZBETH INELVA LUNA COLULA,16,20085.0,4500.0,875.00
9,GARCIA RUIZ ALEXIS,5,9434.0,4485.0,313.95


In [68]:
pagos_totales_brad2 = pd.pivot_table(pagos_brad2,index=['TIPO_ASIGANCION'],values=['MONTO','Facturacion'],aggfunc=['count',np.sum],margins=True)
pagos_totales_brad2 = pd.DataFrame(pagos_totales_brad2.to_records())
pagos_totales_brad2 = pagos_totales_brad2.iloc[:,[0,1,4,3]]
pagos_totales_brad2.columns = ['Division','NumeroPagos','SumaPagos','SumaFacturacion']
pagos_totales_brad2.fillna(0,inplace=True)
pagos_totales_brad2[['SumaPagos','SumaFacturacion']] = pagos_totales_brad2[['SumaPagos','SumaFacturacion']].applymap("${0:,.2f}".format)
pagos_totales_brad2

,Division,NumeroPagos,SumaPagos,SumaFacturacion
0,C0 4PV,25,"$56,031.00","$3,922.17"
1,C0 5PV,10,"$8,209.00",$738.81
2,C0 6PV,8,"$12,240.00","$1,224.00"
3,C1,28,"$38,004.24","$6,460.72"
4,C2,8,"$8,449.00","$1,605.31"
5,C3,21,"$22,699.00","$4,766.79"
6,C4,7,"$8,403.00","$1,932.69"
7,All,107,"$154,035.24","$20,650.49"


# Banco Azteca

# Reporte diario Banco Azteca

In [45]:
#Hacemos la consulta referente a las gestiones de Banco Azteca
sql_gest_baz = cursor.callproc('baz_rpt_gestiones_detallado', [today,today])
for result in cursor.stored_results():
     gestion_baz = pd.DataFrame(result.fetchall())
gestion_baz.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_baz = gestion_baz.loc[gestion_baz['dictamen']=='PROMESA']
#gestion_baz.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion_resultado,fecha_promesa,monto_promesa,comentarios
237,9284762,7,1,1-1-9661-108923,ALBA LUCIA MATUZ PEREZ,9614553755,casa,2020-12-05 10:57:09,HEBALTAZARAL,HERNANDEZ BALTAZAR ALFREDO,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-11,307.00,9614553755 9613329771 Man pagos sem de 30740sem
239,9284753,7,1,1-1-2066-23650,MARCO ANTONIO MARTINEZ SOTO,5511749466,celular,2020-12-05 11:30:19,HEBALTAZARAL,HERNANDEZ BALTAZAR ALFREDO,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-06,100.00,5511749466 EL CONVENIO ESTA ROTO SE LE HACE NO...
241,9284750,7,1,1-20-1675-4882,JAVIER TREJO RAMIREZ,5538066228,celular,2020-12-05 11:34:01,HEBALTAZARAL,HERNANDEZ BALTAZAR ALFREDO,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-08,359.00,5538066228 5556181003 SE REPORTA PAGO PARA AGE...


In [46]:
asignacion_baz = pd.read_csv(''+ruta_asig_baz+'/Asignacion banco azteca '+str(mes2)+'.csv')
asignacion_baz = asignacion_baz.rename(columns=str.lower)
#|asignacion_baz.head(3)

,#fiidcampana,fipais,ficanal,fisucursal,fifolio
0,164,1,1,5496,4183
1,164,1,1,110,52389
2,164,1,1,5254,28682


In [47]:
asignacion_baz['credito'] = asignacion_baz['fipais'].map(str)+'-'+asignacion_baz['ficanal'].map(str)+'-'+asignacion_baz['fisucursal'].map(str)+'-'+asignacion_baz['fifolio'].map(str)
#asignacion_baz.head(3)

,#fiidcampana,fipais,ficanal,fisucursal,fifolio,credito
0,164,1,1,5496,4183,1-1-5496-4183
1,164,1,1,110,52389,1-1-110-52389
2,164,1,1,5254,28682,1-1-5254-28682


In [48]:
consolidado_baz = pd.merge(gestion_baz,asignacion_baz,how='left',on='credito')
cols = ['monto_promesa','#fiidcampana']
consolidado_baz[cols] = consolidado_baz[cols].apply(pd.to_numeric, errors='coerce')
consolidado_baz = consolidado_baz[(consolidado_baz['monto_promesa']>50)]
consolidado_baz = consolidado_baz[(consolidado_baz['monto_promesa']<500000)]
consolidado_baz.fillna('COBRANZA',inplace=True)
consolidado_baz.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion_resultado,fecha_promesa,monto_promesa,comentarios,#fiidcampana,fipais,ficanal,fisucursal,fifolio
0,9284762,7,1,1-1-9661-108923,ALBA LUCIA MATUZ PEREZ,9614553755,casa,2020-12-05 10:57:09,HEBALTAZARAL,HERNANDEZ BALTAZAR ALFREDO,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-11,307.0,9614553755 9613329771 Man pagos sem de 30740sem,COBRANZA,COBRANZA,COBRANZA,COBRANZA,COBRANZA
1,9284753,7,1,1-1-2066-23650,MARCO ANTONIO MARTINEZ SOTO,5511749466,celular,2020-12-05 11:30:19,HEBALTAZARAL,HERNANDEZ BALTAZAR ALFREDO,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-06,100.0,5511749466 EL CONVENIO ESTA ROTO SE LE HACE NO...,COBRANZA,COBRANZA,COBRANZA,COBRANZA,COBRANZA
2,9284750,7,1,1-20-1675-4882,JAVIER TREJO RAMIREZ,5538066228,celular,2020-12-05 11:34:01,HEBALTAZARAL,HERNANDEZ BALTAZAR ALFREDO,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-08,359.0,5538066228 5556181003 SE REPORTA PAGO PARA AGE...,COBRANZA,COBRANZA,COBRANZA,COBRANZA,COBRANZA


In [51]:
#Vemos las promesas reales eliminando duplicados
consolidado_baz2 = consolidado_baz.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa','usuario'])
prom_baz_gestor = pd.pivot_table(consolidado_baz2,index=['nombre_usuario'],values=['unegocio_id','monto_promesa'],aggfunc=['count',np.sum],margins=True)
prom_baz_gestor = pd.DataFrame(prom_baz_gestor.to_records())
prom_baz_gestor = prom_baz_gestor.iloc[:,[0,2,3]]
prom_baz_gestor.columns = ['Dictamen','NumeroPromesas','SumaPromesas']
prom_baz_gestor.fillna(0,inplace=True)
#prom_baz_gestor[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = prom_baz_gestor[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
prom_baz_gestor

,Dictamen,NumeroPromesas,SumaPromesas
0,ALVARADO HERNANDEZ ADRIANA,2,2336.0
1,BRENDA EVELYN CARDOSA SEVEROÂ,4,3398.0
2,GALVAN ESTRADA VICTOR ALFONSO,1,232.0
3,HERNANDEZ BALTAZAR ALFREDO,9,7306.0
4,HUGO GERARDO CASTRO GARCIA,4,15042.0
5,JIMMY ZEPEDA ESPINOSA,1,15025.0
6,LEON SALINAS SUSANA,3,2647.0
7,MARIA ELENA LOPEZ MORA,1,200.0
8,All,25,46186.0


# Acumulado Mes Banco Azteca

In [55]:
writer = pd.ExcelWriter(''+ruta_guardar+'/Reporte Avance Gestor '+now+'.xlsx', engine='xlsxwriter')
prom_liv_gestor.to_excel(writer,'PromDiariaLiv',index=False,header=True)
prom_liv_gestor_mes.to_excel(writer,'PromMesLiv',index=False,header=True)
pagos_totales_liv.to_excel(writer,'PagosTotalesLiv',index=False,header=True)
prom_brad_gestor.to_excel(writer,'PromDiariaBrad',index=False,header=True)
prom_brad_gestor_mes.to_excel(writer,'PromMesBrad',index=False,header=True)
pagos_totales_brad2.to_excel(writer,'PagosTotalesBrad',index=False,header=True)
pivot_baz2.to_excel(writer,'PromDiariaBAZ',index=False,header=True)

writer.save()
writer.close()

In [118]:
writer = pd.ExcelWriter('/home/estadistico/Documents/Erick/Reportes diarios/Pagos duplicados.xlsx', engine='xlsxwriter')
pagos_duplicados.to_excel(writer,'Pagos duplicados Liv',index=False,header=True)

writer.save()
writer.close()

In [56]:
gestion_liv_mes.to_csv('/home/estadistico/Documents/Erick/Reportes diarios/Promesas mes liver.csv',index=False)